## This will be our initial attempt to develop a script to scrape the website of choice.

In [16]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse


def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))
            

"""
 "main" is the function that is actually scraping and downloading the images off a specified website
 input: a url and a directory path
 output/result: images are downloaded into specific folder
 everything above is used in main
"""           
def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each image, download it
        download(img, path)

"""
example of how to run main to download images      
main("https://www.csulb.edu/college-of-education", "C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder")
"""

'\nexample of how to run main to download images      \nmain("https://www.csulb.edu/college-of-education", "C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder")\n'

In [1]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

"""
 "main" is the function that is actually scraping and downloading the images off a specified website
 input: a url and a directory path
 output/result: images are downloaded into specific folder
 everything above is used in main
"""           
def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each image, download it
        download(img, path)

"""
example of how to run main to download images      
main("https://www.csulb.edu/college-of-education", "C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder")
"""

/Users/dslc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'\nexample of how to run main to download images      \nmain("https://www.csulb.edu/college-of-education", "C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder")\n'

In [3]:
from bs4 import BeautifulSoup
import requests

def get_embedded_url(url):
    """
    extracts all links embedded in 'url' that contain "college-of-education"
    stores the links in the list 'observed_links'
    """
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)

    # this is creating the list of urls that are embedded in college of ed homepage that we want to look into
    observed_links = []
    for link in soup.find_all('a'):
        if 'college-of-education' in link.get('href'):
            if 'https://www.csulb.edu' in link.get('href'):
                observed_links.append(link.get('href'))
            elif link.get('href')[:1] == '/':
                https_concat = 'https://www.csulb.edu'+(link.get('href'))
                observed_links.append(https_concat)
    observed_links = list(set(observed_links))
    return observed_links

In [4]:
'''
get all links embedded in "https://www.csulb.edu/college-of-education w/ the condition that 
it has "college-of-education" in the url

store the links in a list, "observed_links"
''' 
observed_links = get_embedded_url("https://www.csulb.edu/college-of-education")

In [5]:
# display the list of links generated above, what does it "physically" look like?
display(observed_links)

# display the number of links that were extracted using get_embedded_url
display(len(observed_links))

# count the number of "valid" or executable links in the observed
count = 0
for i in observed_links:
    if is_valid(i):
        count += 1


['https://www.csulb.edu/college-of-education/credential-center/credential-programs',
 'https://www.csulb.edu/college-of-education/outreach-and-recruitment',
 'https://www.csulb.edu/college-of-education/education-specialist-credential-program-escp',
 'https://www.csulb.edu/college-of-education/alumni',
 'https://www.csulb.edu/college-of-education/school-psychology',
 'https://www.csulb.edu/college-of-education/office-of-the-dean/college-and-university-tools',
 'https://www.csulb.edu/college-of-education/assessment-office',
 'https://www.csulb.edu/college-of-education/article/the-winner-of-our-diploma-frame-giveaway',
 'https://www.csulb.edu/college-of-education/admissions-and-how-apply',
 'https://www.csulb.edu/college-of-education/counseling-psychology',
 'https://www.csulb.edu/college-of-education/outreach-and-recruitment/program-information-sessions',
 'https://www.csulb.edu/college-of-education/teacher-librarian-services-credential',
 'https://www.csulb.edu/college-of-education/cred

81

Image Download

The following code block is where I actually begin to download images. The loop goes through the list of interest developed, "observed_links", above and runs "main" to get the image on the url specified (Note: the url specified is inside the list "observed_links")



In [7]:
#takes a lil bit
for i in range(len(observed_links)):
    main(observed_links[i], "/Users/dslc/Documents/observed_links")

Extracting images: 100%|█████████████████████| 11/11 [00:00<00:00, 52428.80it/s]


Extracting images: 100%|█████████████████████| 11/11 [00:00<00:00, 38098.55it/s]


Extracting images: 100%|█████████████████████| 14/14 [00:00<00:00, 44283.75it/s]


Extracting images: 100%|█████████████████████| 10/10 [00:00<00:00, 34239.22it/s]


Extracting images: 100%|█████████████████████| 23/23 [00:00<00:00, 66301.71it/s]


Extracting images: 100%|█████████████████████| 11/11 [00:00<00:00, 35032.15it/s]


Next block is only for inspection purposes,user can skip to the next block

In [6]:
# count the files in the folder that was the target of my downloads
dir_path = "/Users/dslc/Documents/observed_links"
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 117


The process above yields 81 searchable College of Education webpages and 82 image downloads. 1 image is actually repeated 4x so the net total image downloads for the identified webpages is effectively 78.

Next we will try to expand on the list of searchable webpages by running the function "get_embedded_url" for each of the 81 searchable webpages identified above. (Note: The 81 webpages ID'd above are embedded in the College of Education Home page.)

In [7]:
display(observed_links)
display(len(observed_links))

['https://www.csulb.edu/college-of-education/credential-center/credential-programs',
 'https://www.csulb.edu/college-of-education/outreach-and-recruitment',
 'https://www.csulb.edu/college-of-education/education-specialist-credential-program-escp',
 'https://www.csulb.edu/college-of-education/alumni',
 'https://www.csulb.edu/college-of-education/school-psychology',
 'https://www.csulb.edu/college-of-education/office-of-the-dean/college-and-university-tools',
 'https://www.csulb.edu/college-of-education/assessment-office',
 'https://www.csulb.edu/college-of-education/article/the-winner-of-our-diploma-frame-giveaway',
 'https://www.csulb.edu/college-of-education/admissions-and-how-apply',
 'https://www.csulb.edu/college-of-education/counseling-psychology',
 'https://www.csulb.edu/college-of-education/outreach-and-recruitment/program-information-sessions',
 'https://www.csulb.edu/college-of-education/teacher-librarian-services-credential',
 'https://www.csulb.edu/college-of-education/cred

81

In [22]:
#also takes a lil bit
'''
observed_links_lvl2 is an store "level 2" scrape of embedded links.
i.e. for each webpage url in "observed_links" we extract all embedded links that satisfy our criteria.
    - such criteria are provided in the "get_embedded_url" function definition block of code
'''
observed_links_lvl2 = [None]*len(observed_links)
for i in tqdm(range(len(observed_links))):
    observed_links_lvl2[i] = get_embedded_url(observed_links[i])
print(len(observed_links_lvl2))

100%|███████████████████████████████████████████| 81/81 [02:45<00:00,  2.04s/it]

81


Next block is only for inspection purposes,user can skip to the next block

In [ ]:
count = 0
for i in range(len(observed_links_lvl2)):
#     print(len(observed_links_lvl2[i]))
    count += 1
print('this is how much is in observed_links_lvl2')
print(count)

In [24]:
from itertools import chain
lvl2_flat = tqdm(list(chain.from_iterable(observed_links_lvl2)))
#display(lvl2_flat)
print('this is how much is in lvl2 flat')
display(len(lvl2_flat))


  0%|                                                  | 0/6108 [00:10<?, ?it/s]

this is how much is in lvl2 flat


6108

In [36]:
#alphabetical OG

lvl2_flat
display(len(lvl2_flat))
lvl2_flat_unique = (list(sorted(set(lvl2_flat))))
#takes all the lvl2 links and removes any dupes
#display(lvl2_flat_unique)
display(len(lvl2_flat_unique))
display(lvl2_flat_unique)

6108

748

['https://www.csulb.edu/academic-affairs/faculty-affairs/careers-college-of-education',
 'https://www.csulb.edu/article/college-of-education-scholarship-gets-a-financial-boost',
 'https://www.csulb.edu/college-of-education',
 'https://www.csulb.edu/college-of-education-faculty-staff-all',
 'https://www.csulb.edu/college-of-education-research-and-grants/article/dr-trini-lewis-presents-at-cabe-graduate-students',
 'https://www.csulb.edu/college-of-education/about-the-college',
 'https://www.csulb.edu/college-of-education/adapted-physical-education-added-authorization-program-review-launchpad',
 'https://www.csulb.edu/college-of-education/admissions-and-how-apply',
 'https://www.csulb.edu/college-of-education/admissions-and-how-to-apply',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling/asec-job-openings',
 'https://www.csulb.edu/college-of-education/advanced-stud

In [38]:
#alphabetical NEW

import gspread
from oauth2client.service_account import ServiceAccountCredentials

def get_column_as_list(sheet_name, column_name):
    # Replace 'path/to/your/credentials.json' with the actual path to your JSON credentials file.
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/dslc/Downloads/my-project-393419-671dd8d01f16.json', scope)
    
    # Authenticate with the Google Sheets API
    gc = gspread.authorize(credentials)

    # Open the spreadsheet by its name
    sh = gc.open(sheet_name)

    # Select the first sheet (index 0)
    worksheet = sh.get_worksheet(0)

    # Get the column data as a list
    column_list = sorted(worksheet.col_values(ord(column_name) - 64))  # Assuming column name is a single uppercase letter

    return column_list

if __name__ == "__main__":
    # Replace 'Your_Spreadsheet_Name' with the actual name of your Google Sheets spreadsheet.
    # Replace 'A' with the column name you want to convert to a list.
    sheet_name = 'Dr Dev links'
    column_name = 'B'

    column_list = get_column_as_list(sheet_name, column_name)
    display(len(column_list[38:])) 
    display(column_list[38:])


384

['https://ccog.calstate.edu/',
 'https://csulb-my.sharepoint.com/:w:/g/personal/matt_evans_csulb_edu/EQ5xWCl2_cVAuc2r9cB6ISABMIXjAzbbMIe1AoIoaW7Aug?e=JD4lXw',
 'https://csulb-my.sharepoint.com/personal/matt_evans_csulb_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fmatt%5Fevans%5Fcsulb%5Fedu%2FDocuments%2Fced%5Fweb%2Fced%5Fstrategic%5Fpriorities%2Epdf&parent=%2Fpersonal%2Fmatt%5Fevans%5Fcsulb%5Fedu%2FDocuments%2Fced%5Fweb&ga=1',
 'https://www.csulb.edu/college-of-education',
 'https://www.csulb.edu/college-of-education',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling/asec-department-documents',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling/faculty-staff',
 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling/frequently-asked-questions-for',
 'https://www.csulb.edu/college-of-education/a

In [14]:
from collections import Counter
lvl2_flat_unique_minus_bs = list((Counter(lvl2_flat_unique)-Counter(observed_links)).elements())
display(len(lvl2_flat_unique_minus_bs))
#gives us all the links excluding the 81 from the homepage

668

In [20]:
main('https://www.csulb.edu/college-of-education','/Users/dslc/Documents/yea')

Extracting images: 100%|█████████████████████| 23/23 [00:00<00:00, 82452.13it/s]


In [21]:
display(count)

81

In [106]:
# for i in range(2):
#      main(lvl2_flat_unique_minus_bs,'/Users/dslc/Documents/allofem-homepage2')

for i in range(len(lvl2_flat_unique_minus_bs)):
    main(lvl2_flat_unique,'/Users/dslc/Documents/allofem-homepage')

InvalidSchema: No connection adapters were found for "['https://www.csulb.edu/college-of-education/common-standard-4-continuous-improvement', 'https://www.csulb.edu/academic-affairs/faculty-affairs/careers-college-of-education', 'https://www.csulb.edu/college-of-education/article/dr-cara-richards-tutor-joins-research-team-to-replicate-and-examine-', 'https://www.csulb.edu/college-of-education/master-of-science-special-education/international-applicants', 'https://www.csulb.edu/college-of-education/advanced-studies-education-and-counseling/page/kelli-sanderson', 'https://www.csulb.edu/college-of-education/curriculum-and-instruction/frequently-asked-questions', 'https://www.csulb.edu/college-of-education/liberal-studies/faculty-staff', 'https://www.csulb.edu/college-of-education/teacher-education/programs-to-become-a-teacher', 'https://www.csulb.edu/college-of-education/faculty-resources-and-support/instructional-practices', 'https://www.csulb.edu/college-of-education/graduate-studies-of

In [92]:
# count the files in the folder that was the target of my downloads
dir_path = "/Users/dslc/Documents/allofem-homepage"
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 54


In [85]:
print(lvl2_flat_unique_minus_bs[667])

https://www.csulb.edu/college-of-education/long-beach-trauma-recovery-center/media-coverage-of-the-long-beach-trauma


In [2]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

set_nb_theme('monokai')